In [ ]:
WITH strategy_vs_curation AS (WITH
        cust_tmp AS (
          SELECT
            id AS customer_id
            ,name AS customer_name
          FROM customers
          WHERE 1=1 -- no filter on 'customers.name'

        ),

        attribute_and_value AS (
          SELECT
            customer_name,
            name as attribute_name,
            regexp_split_to_array(
              regexp_replace(value,
                    '(\[|\"|\])',
                    '',
                    'g'),','
                    ) as nested_value_name
          FROM curated_product_fields
          INNER JOIN cust_tmp
            ON cust_tmp.customer_id = curated_product_fields.customer_id
          WHERE value != 'n/a'
        ),

        unique_attribute_value_curated AS (
          SELECT DISTINCT
            customer_name,
            'curation' AS source,
            attribute_name AS attribute,
            value_name AS value
          FROM attribute_and_value,
          UNNEST(nested_value_name) as value_name
        ),

        unique_attribute_value_strategy AS (
          SELECT DISTINCT
            customer_name,
            'strategy' AS source,
            attributes.snake_case_name AS Attribute,
            values.name AS Value,
            strategy_buckets_attributes."question_type" AS question_type
          FROM public.strategy_versions  AS strategy_versions
          INNER JOIN public.strategy_buckets  AS strategy_buckets ON (strategy_versions."id") = (strategy_buckets."strategy_version_id")
          INNER JOIN public.buckets  AS buckets ON (strategy_buckets."bucket_id") = (buckets."id")
          INNER JOIN public.strategy_buckets_attributes  AS strategy_buckets_attributes ON (strategy_buckets."id") = (strategy_buckets_attributes."strategy_bucket_id")
          INNER JOIN public.attributes  AS attributes ON (strategy_buckets_attributes."attribute_id") = (attributes."id")
          INNER JOIN cust_tmp ON (strategy_versions."customer_id") = (cust_tmp.customer_id)
          LEFT JOIN (SELECT * FROM public.strategy_buckets_attributes_values WHERE status = 'OPT_IN') AS strategy_buckets_attributes_values ON (strategy_buckets_attributes."id") = (strategy_buckets_attributes_values."strategy_buckets_attribute_id")
          LEFT JOIN public.values AS values ON (strategy_buckets_attributes_values."value_id") = (values."id")
          WHERE (strategy_versions."active" ) = True
            AND (strategy_buckets."active") = True
            AND (strategy_buckets_attributes."status" ) = 'OPT_IN'
          GROUP BY
              1,
              2,
              3,
              4,
              5
        ),

        full_joins As (
          SELECT DISTINCT
            customer_name,
            CASE
            WHEN (uavs.source IS NOT NULL AND uavc.source IS NOT NULL) THEN 'Strategy & Curation'
            ELSE coalesce(uavs.source, uavc.source) END AS source
            ,attribute
            ,value
          FROM unique_attribute_value_strategy AS uavs
           FULL OUTER JOIN unique_attribute_value_curated AS uavc
            USING(customer_name, attribute,value)
        )

        SELECT DISTINCT
          qt.question_type,
          full_joins.*
        FROM full_joins
        LEFT JOIN unique_attribute_value_strategy AS qt
          USING(attribute)
        WHERE full_joins.value IS NOT NULL
        ORDER BY attribute ASC , full_joins.value ASC
       )
SELECT * FROM (
SELECT *, DENSE_RANK() OVER (ORDER BY z___min_rank) as z___pivot_row_rank, RANK() OVER (PARTITION BY z__pivot_col_rank ORDER BY z___min_rank) as z__pivot_col_ordering, CASE WHEN z___min_rank = z___rank THEN 1 ELSE 0 END AS z__is_highest_ranked_cell FROM (
SELECT *, MIN(z___rank) OVER (PARTITION BY "strategy_vs_curation.attribute","strategy_vs_curation.question_type","strategy_vs_curation.value") as z___min_rank FROM (
SELECT *, RANK() OVER (ORDER BY "strategy_vs_curation.attribute" ASC, z__pivot_col_rank, "strategy_vs_curation.question_type", "strategy_vs_curation.value") AS z___rank FROM (
SELECT *, DENSE_RANK() OVER (ORDER BY CASE WHEN "strategy_vs_curation.source" IS NULL THEN 1 ELSE 0 END, "strategy_vs_curation.source") AS z__pivot_col_rank FROM (
SELECT
    strategy_vs_curation."source"  AS "strategy_vs_curation.source",
    strategy_vs_curation."attribute"  AS "strategy_vs_curation.attribute",
    strategy_vs_curation."question_type"  AS "strategy_vs_curation.question_type",
    strategy_vs_curation."value"  AS "strategy_vs_curation.value",
    COUNT(*) AS "strategy_vs_curation.count"
FROM strategy_vs_curation
WHERE (strategy_vs_curation."question_type" ) = 'multiselect' AND ((( strategy_vs_curation."attribute"  ) NOT LIKE 'q^_ml^_%' ESCAPE '^' AND ( strategy_vs_curation."attribute"  ) NOT IN ('capacity_lbs', 'width_in', 'depth_in', 'length_in', 'height_in') OR ( strategy_vs_curation."attribute"  ) IS NULL))
GROUP BY
    1,
    2,
    3,
    4) ww
) bb WHERE z__pivot_col_rank <= 16384
) aa
) xx
) zz
 WHERE (z__pivot_col_rank <= 50 OR z__is_highest_ranked_cell = 1) AND (z___pivot_row_rank <= 5000000000000 OR z__pivot_col_ordering = 1) ORDER BY z___pivot_row_rank